In [5]:
!pip install colorama
from datetime import datetime
import colorama

colorama.init(autoreset=True)

def log(message, level="normal"):
    """
    Выводит сообщение с временной меткой и цветом, зависящим от уровня важности.

    Аргументы:
        message (str): Сообщение для вывода.
        level (str): Уровень важности: 'info', 'warning', 'error', 'debug' или 'normal'.
    """
    timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]

    if level == "info":
        color = colorama.Fore.GREEN
    elif level == "warning":
        color = colorama.Fore.YELLOW
    elif level == "error":
        color = colorama.Fore.RED
    elif level == "debug":
        color = colorama.Fore.BLUE
    else: # normal
        color = colorama.Fore.WHITE

    print(f"{timestamp} {color}{message}{colorama.Style.RESET_ALL}")

In [6]:
!pip install transitions
import random
import threading
import time
from transitions.extensions import GraphMachine


states = [
    'off',
    'on',
    'eco_active',
    'eco_pause',
    'fan_active',
    'fan_pause',
    'heating_active',
    'heating_pause',
    'cooling_active',
    'cooling_pause',
    'stop',
    'broken'
]

transitions_list = [
    # Переходы питания.
    {'trigger': 'power_on', 'source': 'off', 'dest': 'on'},
    {'trigger': 'power_off', 'source': ['on', 'stop', 'broken'], 'dest': 'off'},

    # Переходы из состояния 'on' в рабочий режим.
    {'trigger': 'set_eco', 'source': 'on', 'dest': 'eco_active'},
    {'trigger': 'set_fan', 'source': 'on', 'dest': 'fan_active'},
    {'trigger': 'set_heating', 'source': 'on', 'dest': 'heating_active'},
    {'trigger': 'set_cooling', 'source': 'on', 'dest': 'cooling_active'},

    # По истечении таймера работы активные состояния переходят в 'stop'.
    {'trigger': 'timeout', 'source': ['eco_active', 'fan_active', 'heating_active', 'cooling_active'], 'dest': 'stop'},

    # Переходы на паузу из активных режимов в состояние 'pause'.
    {'trigger': 'pause', 'source': 'eco_active', 'dest': 'eco_pause'},
    {'trigger': 'pause', 'source': 'fan_active', 'dest': 'fan_pause'},
    {'trigger': 'pause', 'source': 'heating_active', 'dest': 'heating_pause'},
    {'trigger': 'pause', 'source': 'cooling_active', 'dest': 'cooling_pause'},

    # Переходы возобновления работы из состояния паузы в активное состояние.
    {'trigger': 'resume', 'source': 'eco_pause', 'dest': 'eco_active'},
    {'trigger': 'resume', 'source': 'fan_pause', 'dest': 'fan_active'},
    {'trigger': 'resume', 'source': 'heating_pause', 'dest': 'heating_active'},
    {'trigger': 'resume', 'source': 'cooling_pause', 'dest': 'cooling_active'},

    # Случайный сбой может произойти в любом активном рабочем состоянии.
    {'trigger': 'random_break', 'source': ['eco_active', 'fan_active', 'heating_active', 'cooling_active'], 'dest': 'broken'},

    # Ремонт неисправного кондиционера.
    {'trigger': 'repair', 'source': 'broken', 'dest': 'on'},

    # Перезапуск после остановки.
    {'trigger': 'restart', 'source': 'stop', 'dest': 'on'},
]


class AC(GraphMachine):
    def __init__(self):
        super(AC, self).__init__(states=states, transitions=transitions_list, initial='off', auto_transitions=False)

        # Дескрипторы таймеров (используются в рабочих режимах).
        self._working_timer = None
        self._random_break_timer = None

    def _cancel_timers(self):
        if self._working_timer is not None:
            self._working_timer.cancel()
            self._working_timer = None
        if self._random_break_timer is not None:
            self._random_break_timer.cancel()
            self._random_break_timer = None

    # --- Колбэки для входа в активные рабочие состояния ---
    def on_enter_eco_active(self):
        log("Переход в режим ЭКО.")
        self.start_working_timer()
        self.start_random_break_timer()

    def on_enter_fan_active(self):
        log("Переход в режим ВЕНТИЛЯТОР.")
        self.start_working_timer()
        self.start_random_break_timer()

    def on_enter_heating_active(self):
        log("Переход в режим ОБОГРЕВ.")
        self.start_working_timer()
        self.start_random_break_timer()

    def on_enter_cooling_active(self):
        log("Переход в режим ОХЛАЖДЕНИЕ.")
        self.start_working_timer()
        self.start_random_break_timer()

    # --- Колбэки для выхода из активных рабочих состояний ---
    def on_exit_eco_active(self):
        log("Выход из режима ЭКО.")
        self._cancel_timers()

    def on_exit_fan_active(self):
        log("Выход из режима ВЕНТИЛЯТОР.")
        self._cancel_timers()

    def on_exit_heating_active(self):
        log("Выход из режима ОБОГРЕВ.")
        self._cancel_timers()

    def on_exit_cooling_active(self):
        log("Выход из режима ОХЛАЖДЕНИЕ.")
        self._cancel_timers()

    # --- Колбэки для входа в состояние паузы ---
    def on_enter_eco_pause(self):
        log("Режим ЭКО на паузе.")
        self._cancel_timers()

    def on_enter_fan_pause(self):
        log("Режим ВЕНТИЛЯТОР на паузе.")
        self._cancel_timers()

    def on_enter_heating_pause(self):
        log("Режим ОБОГРЕВ на паузе.")
        self._cancel_timers()

    def on_enter_cooling_pause(self):
        log("Режим ОХЛАЖДЕНИЕ на паузе.")
        self._cancel_timers()

    # --- Колбэк для состояния 'stop' ---
    def on_enter_stop(self):
        log("Таймер работы истек. Кондиционер остановлен.")
        self._cancel_timers()

    # --- Колбэк для состояния 'broken' ---
    def on_enter_broken(self):
        log("Кондиционер вышел из строя! Пожалуйста, отремонтируйте его.")
        self._cancel_timers()

    # --- Колбэки для состояний включено/выключено ---
    def on_enter_on(self):
        log("Кондиционер включен.")

    def on_enter_off(self):
        log("Кондиционер выключен.")
        self._cancel_timers()

    # --- Управление таймером для рабочих состояний ---
    def start_working_timer(self, duration=10):
        """Запускает таймер, который вызывает переход по истечении фиксированной продолжительности."""
        log("Запуск таймера работы на {} секунд.".format(duration))
        self._working_timer = threading.Timer(duration, self._on_timeout)
        self._working_timer.start()

    def _on_timeout(self):
        if self.state in ['eco_active', 'fan_active', 'heating_active', 'cooling_active']:
            log("Таймер работы истек. Остановка.")
            self.timeout()

    # --- Управление таймером случайного сбоя ---
    def start_random_break_timer(self, min_duration=5, max_duration=15):
        """Запускает таймер, который вызовет случайный сбой после случайной задержки."""
        duration = random.uniform(min_duration, max_duration)
        log("Запуск таймера случайного сбоя на {:.2f} секунд.".format(duration))
        self._random_break_timer = threading.Timer(duration, self._on_random_break)
        self._random_break_timer.start()

    def _on_random_break(self):
        if self.state in ['eco_active', 'fan_active', 'heating_active', 'cooling_active']:
            log("Произошел случайный сбой! Кондиционер вышел из строя.")
            self.random_break()


In [7]:
ac = AC()

log(f"Начальное состояние: {ac.state}", level="info")
ac.power_on()
log("Состояние после включения: {}".format(ac.state), level="info")

ac.set_cooling()
log("Состояние после установки режима охлаждения: {}".format(ac.state), level="info")

time.sleep(3)
ac.pause()
log("Состояние после постановки на паузу: {}".format(ac.state), level="info")

time.sleep(2)
ac.resume()
log("Состояние после возобновления работы: {}".format(ac.state), level="info")

log("Ожидание срабатывания таймеров (истечение времени или случайный сбой)...", level="info")
time.sleep(20)
if ac.state == 'broken':
    ac.repair()
    log("Состояние после ремонта: {}".format(ac.state), level="info")

log("Конечное состояние: {}".format(ac.state), level="info")

12:17:31.865 Начальное состояние: off
12:17:31.867 Кондиционер включен.
12:17:31.869 Состояние после включения: on
12:17:31.871 Переход в режим ОХЛАЖДЕНИЕ.
12:17:31.873 Запуск таймера работы на 10 секунд.
12:17:31.876 Запуск таймера случайного сбоя на 12.24 секунд.
12:17:31.879 Состояние после установки режима охлаждения: cooling_active
12:17:34.881 Выход из режима ОХЛАЖДЕНИЕ.
12:17:34.882 Режим ОХЛАЖДЕНИЕ на паузе.
12:17:34.885 Состояние после постановки на паузу: cooling_pause
12:17:36.888 Переход в режим ОХЛАЖДЕНИЕ.
12:17:36.890 Запуск таймера работы на 10 секунд.
12:17:36.893 Запуск таймера случайного сбоя на 13.06 секунд.
12:17:36.896 Состояние после возобновления работы: cooling_active
12:17:36.899 Ожидание срабатывания таймеров (истечение времени или случайный сбой)...
12:17:46.909 Таймер работы истек. Остановка.
12:17:46.913 Выход из режима ОХЛАЖДЕНИЕ.
12:17:46.916 Таймер работы истек. Кондиционер остановлен.
12:17:56.902 Конечное состояние: stop


In [8]:
!pip install graphviz
import graphviz
ac_model = AC()
ac_model.get_combined_graph().draw('ac_state_diagram.png', prog='dot')
